### Student:
2. Alexander Gorelik

In [1]:
import csv
import random
import sqlite3
from sqlite3 import Error
import pandas as pd
import os
import pyarrow.csv as pv
import pyarrow.parquet as pq
import dask.dataframe as dd
import math

### Create local CSV file “mydata.csv” with 1000000 rows

In [2]:
csv_file = 'mydata.csv'

header = ['id', 'fruit', 'price', 'color']
fruits = ['Orange', 'Grape', 'Apple', 'Banana', 'Pineapple', 'Avocado']
colors = ['Red', 'Green', 'Yellow', 'Blue']

with open(csv_file, 'w', newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)

    for line_num in range(1, 1000000):
        line = [f'{line_num}', f'{random.choice(fruits)}', random.randint(10, 100), f'{random.choice(colors)}']
        csv_writer.writerow(line)

----
# Task 1: CSV and SQL

### 1.1 & 1.2

In [ ]:
df_data = pd.read_csv('mydata.csv')
table_name = 'mydata'
db_name = 'mydb.db'
conn = None
try:
    conn = sqlite3.connect(db_name)
    df_data.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
except Error as e:
    print(e)
finally:
    if conn:
        conn.close()

### 1.3
----
**SELECT** part is a **Project operation** ( operation on columns, selecting columns ) <br>
**WHERE** part is a **Predicate operation** ( operation on rows, selecting rows )

In [ ]:
command_1 = "SELECT fruit, price " \
            "FROM mydata " \
            "WHERE fruit = 'Banana' "

command_2 = "SELECT fruit, color, price " \
            "FROM mydata " \
            "WHERE fruit = 'Avocado' AND color = 'Yellow'"

In [ ]:
conn = None
try:
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    for row in cur.execute(command_2):
        #print(row)
        pass
except Error as e:
    print(e)
finally:
    if conn:
        conn.close()

-----
# Task 2: CSV and Parquet

### 2.1

In [ ]:
lines_num = 0
with open(csv_file) as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        lines_num += 1
print(f'Number of lines in mydata.csv is {lines_num}')

if not os.path.isdir('./parquets'):
    os.mkdir('./parquets')

### 2.2

In [ ]:
table = pv.read_csv(csv_file)
pq.write_table(table, './parquets/mydatapyarrow.parquet')

### 2.3

In [ ]:
df = dd.read_csv(csv_file)
df.to_parquet('./parquets/mydatadask.parquet', write_index=False)

### 2.4

In [ ]:
df_data.to_parquet('./parquets/mydatapandas.parquet')

### 2.5
Dask generate Parquet file differently as dask is a library for parallel computing and working with large datasets.
Dask is a parallel computing framework that makes it easy to convert a lot of CSV files to Parquet files with a
single operation.
A Dask DataFrame contains multiple Pandas DataFrames.
Each Pandas DataFrame is referred to a partition of the Dask DataFrame.
Each partition in the Dask DataFrame is written out to disk in the Parquet file format.
Dask writes out files in parallel so the Parquet files are written simultaneously.
In our case, the Dask DataFrame consisted of one Pandas DataFrames as we have only one CSV file and as a result
mydatadask.parquet directory has only one part.0.parquet
This directory have _common_metadata that the schema of the dataset can be read from
and _metadata that the to_parquet function generated that provides things that also can be got from each
file's footer and scanning all files metadata footers can be painful, especially when we have millions of files.

----

# Task 3: Split CSV files

### 3.1

In [ ]:
file_size = os.path.getsize(csv_file)
middle = int(file_size/2)

print(f'mydata.csv file size is {file_size} bytes')
print(f'middle is {middle}')

### 3.2

In [ ]:
def first_chunk(file_csv, file_mid):
    lines_count = 0
    with open(file_csv, 'rb') as f:
        d = f.read(file_mid).decode(encoding='utf-8')
        lines_count = len(d.split('\n'))
    return lines_count


def last_chunk(file_csv, file_mid):
    lines_count = 0
    columns_count = 0
    with open(file_csv, 'rb') as f:
        f.seek(file_mid, 0)
        d = f.read().decode(encoding='utf-8')
        lines_count = len(d.split('\n'))
    return lines_count-1   # subtracting 1 for automatically generated empty line at the end while creating a csv file

In [ ]:
l1 = first_chunk('mydata.csv', middle)
print(f'number of lines in the first chunk is {l1}')

In [ ]:
l2 = last_chunk('mydata.csv', middle)
print(f'number of lines in the last chunk is {l2}')

In [ ]:
print(f'Total number of lines of 2 chunks is {l1+l2}')

### 3.3

We are getting 1 line more, the reason is when we defining the middle, we split file in to two parts and the
middle is very unlikely will fall exactly at the end of the line. So as a result it "cuts" one line in to two
parts, then we have one part of the line as a line in the first chunk and the second part of a line as a row
in the last chunk. That results in one additional line when we count total lines of two chunks.

### 3.4

In this algorithm we are processing csv file in chunks, the general idea that we reading the chunk, then we check if the last line of chunk is broken, so if yes, we continue reading the line byte by byte until we are reaching new line character so we will have complete rows and also counting these byte steps. Then we count the lines.<br>
When we are getting to the next chunk we use our saved byte steps number we made in previous chunk to ignore these bytes in the new chunk, so we are actually have our rows starting from complete row.<br>
*(We know that we could also use seek() to locate the pointer to ignore when we reading the bytes that we already read at the end of the previous chunk, but we thought that we want to leave seek() like an idea of processing file by chunks with defined size, so using seek again was like splitting the file in different size, so we sticked to our idea of ignore not relevant bytes after reading.* 

In [ ]:
def calculate_total_lines_num(file_csv, chunk_size):
    total_lines_number = 0
    ignore_bytes = 0
    csv_size = os.path.getsize(file_csv)
    number_of_chunks = math.ceil(csv_size/chunk_size)
    threshold = chunk_size
    with open(file_csv, 'rb') as f:
        for i in range(0, number_of_chunks):
            f.seek(chunk_size*i, 0)
            d = f.read(chunk_size).decode(encoding='utf-8')
            relevant = d[ignore_bytes:]
            if relevant == '':
                break
            ignore_bytes = 0
            if relevant[-1] != '\n':
                while ignore_bytes <= threshold:
                    ignore_bytes += 1
                    ch = f.read(1).decode(encoding='utf-8')
                    relevant += ch
                    if ch == '\n':
                        break
            lines_count = len(relevant.split('\n'))-1  # since when splitting by \n we get additional '' at the end
            total_lines_number += lines_count
    return total_lines_number

In [ ]:
total_n_of_lines = calculate_total_lines_num('mydata.csv', middle)
print(f'Total number of rows for two chunks is: {total_n_of_lines}')

### 3.5

In [18]:
chunk_size = 11
total_number_of_lines = calculate_total_lines_num('mydata.csv', chunk_size*(1024**2))
print(f'Total number of rows after processing file by 16MB chunks is: {total_number_of_lines}')

Total number of rows after processing file by 16MB chunks is: 1000000
